In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
from compile_sweep_results import compile_sweep_results
from tqdm import tqdm

In [ ]:
root = Path("/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/symmetry_breaking/sweep_results/")
sweep_name = "sweep06_field_tuning"
output_dir = root / sweep_name
fig_path = Path("/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/slides/killifish/20250815/")

In [ ]:
rows = []

for path in tqdm(output_dir.glob("*.json")):
    # First try pandas' JSON reader for Series format
    s = pd.read_json(path, typ="series")
    s = s.copy()
    s["__source_file__"] = path.name
    s["sim_id"] = s["__source_file__"].replace("result_", "").replace(".json", "")
    rows.append(s)

### Compile results into pandas dataframes

In [ ]:
array_cols = ["Activator", "Repressor", "rho"]
skip_cols = ["x", "times"] + array_cols
param_cols = [col for col in rows[0].keys() if col not in ["sim_id"] + skip_cols]
param_cols = ["sim_id"] + param_cols

x_grid = (np.asarray(rows[0]["x"]) - 1500)[:, None]
t_grid = np.asarray(rows[0]["times"])[:, None]
x_cols = [f"x{i:04}" for i in range(len(x_grid))]

param_dfs = []
activator_dfs = []
repressor_dfs = []
rho_dfs = []
skipped = 0
# iterate
for row in tqdm(rows):

    # params
    p_temp = pd.DataFrame(row[param_cols])
    param_dfs.append(p_temp.T)
    # try:
    t_vec = np.asarray(row["times"])[:, None]
    sim_id_vec = np.tile(row["sim_id"], len(t_vec))[:, None]
    # N df
    a_array = np.asarray(row["Activator"])
    a_temp = pd.DataFrame(np.c_[sim_id_vec, t_vec, a_array], columns=["sim_id", "t"] + x_cols)
    activator_dfs.append(a_temp)
    # L df
    r_array = np.asarray(row["Repressor"])
    r_temp = pd.DataFrame(np.c_[sim_id_vec, t_vec, r_array], columns=["sim_id", "t"] + x_cols)
    repressor_dfs.append(r_temp)
    # rho df
    rho_array = np.asarray(row["rho"])
    rho_temp = pd.DataFrame(np.c_[sim_id_vec, t_vec, rho_array], columns=["sim_id", "t"] + x_cols)
    rho_dfs.append(rho_temp)
# except:
    skipped += 1


Nodal_df = pd.concat(activator_dfs, axis=0, ignore_index=True)
Lefty_df = pd.concat(repressor_dfs, axis=0, ignore_index=True)
rho_df = pd.concat(rho_dfs, axis=0, ignore_index=True)
param_df = pd.concat(param_dfs, axis=0, ignore_index=True)

Nodal_df.head()

### Count Nodal peaks

In [ ]:
sigma_N = 10
sigma_L = 10
mu_L = 0.0002275846
mu_N = mu_L * 1.11e-4/0.61e-4

N_factor = sigma_N / mu_N
L_factor = sigma_L / mu_L

# target domain sigma: 250
d_sigma = 250
ref_grid = x_grid.ravel()
# generate calculation masks
mid_mask = np.abs(ref_grid) <= d_sigma
left_mask = ref_grid < -d_sigma
right_mask = ref_grid > d_sigma

In [ ]:
from symmetry_breaking.utilities.helpers import count_nodal_peaks_periodic
T_min = 35000
# get last entris
Nodal_df["t"] = Nodal_df["t"].astype(float)
last_nodal_df = Nodal_df.loc[Nodal_df["t"]>=T_min].drop_duplicates(subset=["sim_id"], keep="last")
# last_nodal_df = Nodal_df.groupby("sim_id").last().reset_index()
# keep only those that ran to completion
# T_min = 35000
# last_nodal_df["t"] = last_nodal_df["t"].values.astype(float)
# last_nodal_df = last_nodal_df.loc[last_nodal_df["t"]>=T_min, :]
last_nodal_df.head()

In [ ]:
n_peak_vec = []
peak_pos_vec = []

h_thresh = 10000
k_thresh = 5000
for sim_id in tqdm(last_nodal_df["sim_id"]):

    # get nodal
    nodal_vec = last_nodal_df.loc[last_nodal_df["sim_id"]==sim_id, x_cols].to_numpy()[0].astype(float)

    # count 
    peaks = count_nodal_peaks_periodic(nodal_vec, height_thresh=h_thresh, k_prom=k_thresh)

    n_peak_vec.append(peaks[0])
    peak_pos_vec.append(peaks[1])
    

In [ ]:
temp_df = pd.DataFrame(last_nodal_df["sim_id"].copy())
temp_df["n_peaks"] = n_peak_vec
pattern_df = param_df.merge(temp_df, how="inner", on="sim_id")
pattern_df["tau_N"] = pattern_df["rate_N"].astype(float)**-1 / 3600
pattern_df["D_L"] = pattern_df["D_L"].astype("float") 

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.nonparametric.smoothers_lowess import lowess
from src.utilities.plot_functions import format_2d_plotly
import os

# Choose your continuous colormap
SCALE = px.colors.sequential.Viridis  # or Plasma, Cividis, Magma, etc.
REVERSE = False                       # set True to invert

# Compute D_L range for normalization
dl_vals = pattern_df["D_L"].astype(float)
dl_min, dl_max = float(dl_vals.min()), float(dl_vals.max())

def color_for_dl(v):
    # normalize to [0,1]
    if dl_max == dl_min:
        t = 0.5
    else:
        t = (float(v) - dl_min) / (dl_max - dl_min)
    if REVERSE:
        t = 1.0 - t
    # sample from the colorscale
    return px.colors.sample_colorscale(SCALE, [t])[0]

# 1) Base scatter with continuous color + colorbar
fig = px.scatter(
    pattern_df,
    x="tau_N", y="n_peaks",
    color="D_L",
    color_continuous_scale=SCALE if not REVERSE else SCALE[::-1],
    log_x=True
)

# 2) Add smoothed line per unique D_L (log-x fit)
for dl, gdf in pattern_df.groupby("D_L"):
    gdf = gdf.dropna(subset=["tau_N", "n_peaks"])
    if len(gdf) < 3:
        continue  # need a few points to smooth
    log_x = np.log10(gdf["tau_N"].values)
    y     = gdf["n_peaks"].values

    smoothed = lowess(y, log_x, frac=0.3, it=0, return_sorted=True)
    smooth_x = 10**smoothed[:, 0]
    smooth_y = smoothed[:, 1]

    fig.add_trace(
        go.Scatter(
            x=smooth_x, y=smooth_y,
            mode="lines",
            line=dict(color=color_for_dl(dl), width=3),
            name=f"D_L={dl} trend",
            showlegend=False,           # avoid duplicate legend entries
            hoverinfo="skip"
        )
    )

# (Optional) put lines behind markers
# fig.data = tuple([tr for tr in fig.data if tr.mode == "lines"] +
#                  [tr for tr in fig.data if tr.mode == "markers"])

fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="Lefty Diffusion Coefficient (μm²/s)",
            side="right"        # or "top", "bottom", "left"
        )
    )
)

fig = format_2d_plotly(fig, axis_labels=["trigger timescale", "number of aggregates"], marker_size=6, font_size=16)

fig.show()
fig.write_image(os.path.join(fig_path, "N_agg_plot.png"), scale=2)

In [ ]:
color_map

In [ ]:
# fig = px.scatter(pattern_df.loc[pattern_df["N_amp"]<10], x="N_delta", y="L_center", color="t", hover_data={"sim_id"})
# fig.show()

In [ ]:
from scipy.stats import binned_statistic_2d
from scipy.interpolate import griddata

def compute_flux_grid(df, grid_size=250, log=False, interp_res=250, 
                      id_col = "sim_id",
                      x_col = "N_delta_n",
                      y_col = "L_center_n"):
    """
    Compute a dense interpolated flux field for streamplot from simulation trajectories.

    Parameters
    ----------
    df : pandas.DataFrame
        Must have columns: sim_id, t, N_delta, L_center
    grid_size : int
        Number of bins for coarse binning before interpolation.
    log : bool
        If True, bin in log10 space for both axes.
    interp_res : int
        Resolution of final interpolated grid for plotting.

    Returns
    -------
    Xi, Yi : 2D arrays
        Meshgrid coordinates (uniform spacing).
    Ui, Vi : 2D arrays
        Interpolated vector field components.
    """
    df = df.copy()
    # id_col = "sim_id"
    # x_col = "N_delta_n"
    # y_col = "L_center_n"

    # Compute per-step deltas
    df["dx"] = df.groupby(id_col)[x_col].shift(-1) - df[x_col]
    df["dy"] = df.groupby(id_col)[y_col].shift(-1) - df[y_col]
    df = df.dropna(subset=["dx", "dy"])

    # Choose coordinates
    if log:
        Xdata = np.log10(df[x_col].to_numpy())
        Ydata = np.log10(df[y_col].to_numpy())
    else:
        Xdata = df[x_col].to_numpy()
        Ydata = df[y_col].to_numpy()

    Udata = df["dx"].to_numpy()
    Vdata = df["dy"].to_numpy()

    # Coarse bin edges
    xbins = np.linspace(Xdata.min(), Xdata.max(), grid_size)
    ybins = np.linspace(Ydata.min(), Ydata.max(), grid_size)

    # Bin-averaged velocity components
    U_avg, _, _, _ = binned_statistic_2d(Xdata, Ydata, Udata, statistic='mean', bins=[xbins, ybins])
    V_avg, _, _, _ = binned_statistic_2d(Xdata, Ydata, Vdata, statistic='mean', bins=[xbins, ybins])

    U_avg = U_avg.T
    V_avg = V_avg.T
    
    # Bin centers
    x_centers = 0.5 * (xbins[:-1] + xbins[1:])
    y_centers = 0.5 * (ybins[:-1] + ybins[1:])
    Xc, Yc = np.meshgrid(x_centers, y_centers, indexing="xy")

    # Prepare valid points for interpolation
    mask = ~np.isnan(U_avg) & ~np.isnan(V_avg)
    points = np.column_stack((Xc[mask], Yc[mask]))
    Uvals = U_avg[mask]
    Vvals = V_avg[mask]

    # Dense uniform interpolation grid
    Xi = np.linspace(Xdata.min(), Xdata.max(), interp_res)
    Yi = np.linspace(Ydata.min(), Ydata.max(), interp_res)
    Xi, Yi = np.meshgrid(Xi, Yi, indexing="xy")

    Ui = griddata(points, Uvals, (Xi, Yi), method='linear', fill_value=0)
    Vi = griddata(points, Vvals, (Xi, Yi), method='linear', fill_value=0)

    return Xi, Yi, Ui, Vi

In [ ]:
from matplotlib import pyplot as plt
X, Y, U, V = compute_flux_grid(pattern_df, grid_size=100, log=False)



In [ ]:
import matplotlib as mpl

plot_indices = np.random.choice(3125, n_plot)
x_col = "N_delta_n"
y_col = "L_center_n"

_, t_rank = np.unique(pattern_df["t"], return_inverse=True)
pattern_df["t_int"] = t_rank
t_filter = pattern_df["t_int"] < 10
x_t = pattern_df.loc[t_filter, x_col].to_numpy()[plot_indices]
y_t = pattern_df.loc[t_filter, y_col].to_numpy()[plot_indices]

speed = np.sqrt(U**2 + V**2)

# Make the plot
with plt.style.context("dark_background"):
    
    # cmap = mpl.cm.get_cmap('inferno')
    # cmap_trunc = mpl.colors.ListedColormap(cmap(np.linspace(0, 0.8, 256)))

    fig, ax = plt.subplots(figsize=(8, 6))
    strm = ax.streamplot(
        X, Y, U, V,
        color="white",
        density=2
    )
    
    # Scatter overlay
    # ax.scatter(x_t, y_t, color='black', edgecolor='k', zorder=3, label='t points')
    
    # Axis labels & title
    ax.set_xlabel("Relative Nodal concentration ($[N_c]$-$[N_p]$)")
    ax.set_ylabel("Central Lefty concentration ($L_c$)")
    ax.set_title("Patterning phase space")
    
    # Colorbar for velocity
    # cbar = fig.colorbar(strm.lines, ax=ax, label="Speed")
    # plt.style.use("dark_background")
    ax.legend()
    plt.show()
    
    fig.savefig(fig_path / "survival_phase_diagram.png", dpi=300, bbox_inches="tight")

In [ ]:
import os 

np.random.seed(137)
frame_folder = fig_path / "phase_frames"
os.makedirs(frame_folder, exist_ok=True)

n_plot = 1500
ui, ni = np.unique(pattern_df["sim_id"], return_counts=True)
good_ids = ui[ni==121]

x0 = pattern_df.loc[pattern_df["t_int"] == 0, x_col]
y0 = pattern_df.loc[pattern_df["t_int"] == 0, y_col]
s0 = pattern_df.loc[pattern_df["t_int"] == 0, "sim_id"]
high_ids = s0[(x0>=0.01) & (y0>=0.01)]

# len(good_ids)
# options = np.where(pattern_df["sim_id"].isin(good_ids))[0]
plot_ids = np.random.choice(good_ids, n_plot)
id_filter = pattern_df["sim_id"].isin(plot_ids) & pattern_df["sim_id"].isin(high_ids)

for t in tqdm(pattern_df["t_int"].unique()):
    
    t_filter = (pattern_df["t_int"] == t) & id_filter
    x_t = pattern_df.loc[t_filter, x_col].to_numpy()
    y_t = pattern_df.loc[t_filter, y_col].to_numpy()
    
    speed = np.sqrt(U**2 + V**2)
    
    # Make the plot
    with plt.style.context("dark_background"):
        fig, ax = plt.subplots(figsize=(8, 6))
        strm = ax.streamplot(
            X, Y, U, V,
            color="white",
            # cmap='viridis',
            density=2
        )
        
        # Scatter overlay
        ax.scatter(x_t, y_t, color="#1E90FF", edgecolor='k', zorder=3)
        
        # Axis labels & title
        ax.set_xlabel("Relative Nodal concentration ($[N_c]$-$[N_p]$)")
        ax.set_ylabel("Central Lefty concentration ($L_c$)")
        ax.set_title("Patterning phase space")
        
        # Colorbar for velocity
        # cbar = fig.colorbar(strm.lines, ax=ax, label="Speed")
        
        # ax.legend()
    
        # ax.set_xscale("log")  # for x-axis
        # ax.set_yscale("log")  # for y-axis
        # plt.show()
        
        fig.savefig(frame_folder / f"phase_frame{t:04}.png", dpi=300, bbox_inches="tight")

        plt.close()

In [ ]:
len(high_ids)

In [ ]:
pattern_df.shape

In [ ]:
param_df.shape

In [ ]:
121*3463